In [1]:
from jupyter_dash import JupyterDash
from dash import Dash, html,dcc,Input, Output
import dash_leaflet as dl
import json
import httpx
from rio_tiler.io import COGReader
import xarray as xr

In [2]:
titiler_endpoint = "https://mfkh268xng.execute-api.us-east-1.amazonaws.com/"  # personal endpoint. Please be kind.
#url = "https://rapidepe123.s3.eu-west-2.amazonaws.com/GBR_Wind_Speed_100m_ZSTD.tif"
url = "https://rapidepe123.s3.eu-west-2.amazonaws.com/GBR_wind-speed_100m_COGv2.tif"


In [3]:
with COGReader(url) as cog:
    info = cog.info()
print(info.dict(exclude_none=True))

{'bounds': BoundingBox(left=-14.888427317807158, bottom=47.968268747735536, right=3.399072682193749, top=63.888268747736326), 'minzoom': 4, 'maxzoom': 9, 'band_metadata': [('1', {})], 'band_descriptions': [('1', '')], 'dtype': 'float32', 'nodata_type': 'Nodata', 'colorinterp': ['gray'], 'width': 7315, 'overviews': [2, 4, 8, 16, 32, 64], 'count': 1, 'height': 6368, 'nodata_value': nan, 'driver': 'GTiff'}


In [4]:
with COGReader(url) as cog:
    stats = cog.statistics()

    # assert isinstance(meta, dict)
    # print(list(meta))
    # print(meta["1"].dict())

stat_dict = stats["1"].dict()

minv = stat_dict['min']
maxv = stat_dict['max']
print(minv)
print(maxv)

4.905625820159912
16.434234619140625


In [5]:
def get_point_value(lat,lon):

    with COGReader(url) as cog:
        values = cog.point(lon,lat)
        
    return "{:.2f}".format(float(values[0]))

x = get_point_value(55.5,-4)
print(x)

10.10


In [6]:
r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "rescale": f"{minv},{maxv}",
        "colormap_name": "viridis"
    }
).json()


In [7]:
app = JupyterDash(__name__)

# app.layout = html.Div([
#     dl.Map([dl.TileLayer(), dl.GeoTIFFOverlay( interactive=True, url=geotiff_url, band=0, opacity=0.9)],
#            id="map", style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})])
#app.layout = html.Div([dcc.Graph(figure=fig)])
# color_domain = dict(domainMin=20, domainMax=40, colorscale=['white', 'orange', 'red'])
# color_domain2 = dict(min=20, max=40, colorscale=['white', 'orange', 'red'])
# GEOTIFF_ID = "geotiff-id"
# GEOTIFF_MARKER_ID = "geotiff-marker-id"
# def get_info(feature=None):
#     header = [html.H4("Wind Speed")]
#     if not feature:
#         return header + [html.P("Click for Wind )]
#     return header + [html.B(id="WS"), html.Br(),
#                      "{:.3f} people / mi".format(feature["properties"]["density"]), html.Sup("2")]

# Create info control.
info = html.Div( id="info", className="info",
                style={"position": "absolute", "bottom": "10px", "left": "10px", "z-index": "1000"})



IDs = ["TileMap","WindSpeed@100m"]
app.layout = html.Div([
dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[55, -4],
               zoom=5,
               id = "map",
               children=[
            dl.LayersControl([
                
                   dl.Overlay(dl.LayerGroup(dl.TileLayer(url="https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_nolabels/{z}/{x}/{y}.png",id="TileMap")),name="BaseMap",checked=True),
                   dl.Overlay(dl.LayerGroup(dl.TileLayer(url=r["tiles"][0], opacity=0.8,id="WindSpeed@100m")),name="WS@100m",checked=True),
                   dl.LayerGroup(id="layer"),
                                
                   dl.Colorbar(colorscale="viridis", width=20, height=150, min=minv, max=maxv,unit='m/s',position="bottomright"),
                   info,
                   
])
])
])

@app.callback(Output("info", "children"), [Input("map", "click_lat_lng")])
def map_click(click_lat_lng):

   

    lat= click_lat_lng[0]
    lon=click_lat_lng[1]
    
  
    

    return get_point_value(lat,lon)



app.run_server(debug=True,mode='inline')


The below code looks at leafmap package 

In [8]:
dataDIR = r"C:\Users\mdavi\OneDrive\Documents\GitHub\Dash_Riotiler_Titiler\Resource_Files\GBR_wind-speed_100m_Rast3.nc"
DS = xr.open_dataset(dataDIR)
DS

<xarray.Dataset>
Dimensions:  (lon: 7315, lat: 6368)
Coordinates:
  * lon      (lon) float64 -14.89 -14.88 -14.88 -14.88 ... 3.393 3.395 3.398
  * lat      (lat) float64 63.89 63.88 63.88 63.88 ... 47.98 47.97 47.97 47.97
Data variables:
    WS       (lat, lon) float32 ...
Attributes:
    Conventions:      CF-1.0
    Source_Software:  Esri ArcGIS

In [34]:
# x = DS.assign(Dir1 = (DS["WS"]-DS["WS"]) +7,
#                 Dir2=(DS["WS"]-DS["WS"]) +8,
#                 Dir3 = (DS["WS"]-DS["WS"]) + 10,
#                 Dir4 = (DS["WS"]-DS["WS"]) + 5,
#                 Dir5= (DS["WS"]-DS["WS"]) + 5,
#                 Dir6 = (DS["WS"]-DS["WS"]) + 5,
#                 Dir7 = (DS["WS"]-DS["WS"]) + 15,
#                 Dir8 = (DS["WS"]-DS["WS"]) + 15, 
#                 Dir9 = (DS["WS"]-DS["WS"]) + 9, 
#                 Dir10 = (DS["WS"]-DS["WS"]) + 7,
#                 Dir11 = (DS["WS"]-DS["WS"]) + 8,
#                 Dir12 = (DS["WS"]-DS["WS"]) + 6
# 

import numpy as np
# x = DS.assign({'freq':(['Sector'], [7,8,10,5,5,5,15,15,9,7,8,6], 
#                          {'units': 'deg', 
#                           'long_name':'Direction Sector'})})

# define coordinates
Sector = np.array([0,1,2,3,4,5,6,7,8,9,10,11])

lat = DS['lat'].values
lon = DS['lon'].values
freq = np.array([7,8,10,5,5,5,15,15,9,7,8,6])

fq = xr.DataArray(freq, coords={ 'lat':lat,'lon':lon,'Sector': Sector},
              dims=['lat','lon','Sector'])

DS['freq'] = fq

DS


ValueError: different number of dimensions on data and dims: 1 vs 3

In [10]:
x

<xarray.Dataset>
Dimensions:  (lon: 7315, lat: 6368, Sector: 12)
Coordinates:
  * lon      (lon) float64 -14.89 -14.88 -14.88 -14.88 ... 3.393 3.395 3.398
  * lat      (lat) float64 63.89 63.88 63.88 63.88 ... 47.98 47.97 47.97 47.97
Dimensions without coordinates: Sector
Data variables:
    WS       (lat, lon) float32 ...
    freq     (Sector) int32 7 8 10 5 5 5 15 15 9 7 8 6
Attributes:
    Conventions:      CF-1.0
    Source_Software:  Esri ArcGIS

In [11]:
x.to_netcdf(r"C:\Users\mdavi\OneDrive\Documents\GitHub\Dash_Riotiler_Titiler\Resource_Files\GBR_wind-speed_100m_testDir.nc", "w",encoding={ "gzip", "compression_opts": 9})

SyntaxError: invalid syntax (839124019.py, line 1)

In [ ]:
b = x.sel(lon=[-4], lat=[55], method="nearest")

In [ ]:
df = b.to_dataframe()
df = df.T
df


lon,-3.999677
lat,54.999519
WS,7.465062
Dir1,7.000000
Dir2,8.000000
Dir3,10.000000
Dir4,5.000000
Dir5,5.000000
Dir6,5.000000
Dir7,15.000000
Dir8,15.000000


In [ ]:
df_dir_only = df.drop(['WS'])
df_dir_only.columns = ["Freq"]
df_dir_only

,Freq
Dir1,7.0
Dir2,8.0
Dir3,10.0
Dir4,5.0
Dir5,5.0
Dir6,5.0
Dir7,15.0
Dir8,15.0
Dir9,9.0
Dir10,7.0


In [ ]:
import plotly.express as px
fig = px.bar_polar(df_dir_only,r="Freq")
fig.show()